In [14]:
from tqdm import tqdm  # Importar a biblioteca tqdm
from numpy.polynomial import Polynomial
from tqdm.notebook import tqdm_notebook

def update_cash_flows(df, option_times, options):
    updated_cash_flows = df.copy()
    exercised_options = pd.DataFrame(index=df.index, columns=df.columns, data='-')
    
    #for time in tqdm(option_times, desc="Processing times", unit="time"):
    for time in tqdm_notebook(option_times, desc="Options times", unit="time"):
        ###################################################################################################
        # LSM
        cash_year_temp = []
        for index in df.index:
            cash_year_temp.append(float(updated_cash_flows.loc[index, time]))
        pv_temp = present_value(updated_cash_flows.loc[:, time + 1:], dr).values.tolist()
        
        #p = Polynomial.fit(cash_year_temp, pv_temp, 2)  # regression between (present_value_future_cash_flow) and (cash_flow_year_option)
        p = polynomial_regressions(cash_year_temp, pv_temp, degree, poly_type="poly")
        
        expectation_cf = p(cash_year_temp)  # E[Y|X]

        ###################################################################################################
        i  = 0
        for simulation in df.index:
        #for simulation in tqdm_notebook(df.index, desc="Simulations of the paths", unit="time"):

            if exercised_options.at[simulation, time] == 'n/a':
                continue

            divest_exercised = 'divest' in exercised_options.loc[simulation, :time].values
            if divest_exercised:
                exercised_options.at[simulation, time] = 'n/a'
                i += 1
                continue
                   
            available_options = ['continue', 'divest']
            if 'buyout' not in exercised_options.loc[simulation, :time].values:
                available_options.append('buyout')

            #################################################################################################
            # OPTIONS
            # options = {'continue': [0, 0], 'buyout': [40, 25], 'divest': [0, 100]}
            #           {'option': [expense, reward]}
            
            options_= list(options.keys()) # ['continue', 'buyout', 'divest']
            val_ = list(options.values())  # [[0, 0], [40, 25], [0, 100]]
            
            #'continue':
            cf_1 = cash_year_temp[i] + expectation_cf[i] - val_[0][0] + val_[0][1]
            # 'buyout':
            cf_2 = cash_year_temp[i] + (4/3)*expectation_cf[i] - val_[1][0]
            # 'divest':
            cf_3 = cash_year_temp[i] - val_[2][0] + val_[2][1]
            
            if set(options_) == set(available_options):
                option = {cf_1: 'continue', cf_2: 'buyout', cf_3: 'divest'}[max(cf_1, cf_2, cf_3)]
            else:
                option = {cf_1: 'continue', cf_3: 'divest'}[max(cf_1, cf_3)]
            #################################################################################################
            
            exercised_options.at[simulation, time] = option
            
            # Updating cash flow
            if option == 'buyout':
                updated_cash_flows.at[simulation, time] -= 40
                updated_cash_flows.loc[simulation, time+1:] *= 4/3
            elif option == 'divest':
                updated_cash_flows.at[simulation, time] += 100
                updated_cash_flows.loc[simulation, time+1:] = 0

            i += 1
    return updated_cash_flows, exercised_options

In [ ]:
#updated_cash_flows, exercised_options = update_cash_flows(df1, option_times, options)
updated_cash_flows, exercised_options = update_cash_flows(df1, option_times, options)
df_cf_options, df_options = updated_cash_flows, exercised_options

print(f'Value of the initial investment: ${init_invest:.1f}')
print(f'PV of cash flows: ${np.mean(present_value(df_cf_options, dr)):.1f}')
print(f'Value of the project with options (NPV): ${np.mean(present_value(df_cf_options, dr)) - init_invest:.1f}')
print(f'Option value: ${np.mean(present_value(df_cf_options, dr)) - np.mean(present_value(df1, dr)):.1f}')

#print('\nExercised Options: \n', df_options)